In [1]:
# Import libs
import findspark
findspark.init("/home/antonis/spark-2.3.0-bin-hadoop2.7")

import os.path
import pandas
import math
import time

from metrics.Correlation import Correlation
from IO.Write import Write
from IO.Read import Read
from metrics.RSquare import RSquare
from DataSet.Dataset import DataSet

import pyspark
from pyspark.sql import SparkSession


import sys
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.sql import Row
from pyspark.sql.functions import countDistinct,avg,stddev
from pyspark.ml.feature import (VectorAssembler,VectorIndexer,
                               OneHotEncoder , StringIndexer)
import DataSet.SnpsSelection as s

from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression

In [2]:
from pyspark import SparkConf, SparkContext
conf = (SparkConf()
    .set("spark.driver.maxResultSize", "20g")
       .set('spark_executor_cores',"3"))

#spark = SparkSession.builder.appName('melanoma').getOrCreate()
sc = SparkContext(conf=conf)
spark = SQLContext(sc)

In [3]:
#spark.stop()

# TESTS

In [4]:
#pathSnp = '/media/antonis/red/newSet/maf/maf = 0.05/assoc/pvalue = 0.001/snp1.txt'
#pathSnp = '/media/antonis/red/newSet/maf/maf = 0.05/assoc/pvalue = 0.01/snp2.txt'
pathSnp = '/media/antonis/red/newdata/maf = 0.05/pvalue = 0.001/snp2.txt'

#pathSnp = '/media/antonis/Antonis_Moulopoulos/newSet/pvalue = 0.001/snp1.txt'
#pathSnp = '/media/antonis/Antonis_Moulopoulos/newdata/maf = 0.05/pvalue = 0.001/snp2.txt'

data = spark.read.option("maxColumns", 80000).csv(pathSnp,inferSchema=True,header=True)
data=data.withColumnRenamed('TARGET','label')

In [5]:
features = []
for i in data.columns:
    if 'rs' not in i and i !='label':
        features.append(i)
print(len(features))
#print((features))

141


In [7]:
#data.select('patients','label').show(10)

In [6]:
from pyspark.ml.linalg import DenseVector

d = data.drop('patients')
for i in features:
    d = d.drop(i)
print("columns = ",len(d.columns))
li = len(d.columns)
input_data = d.rdd.map(lambda x: (x[li-1], DenseVector(x[:li-1])))

df = spark.createDataFrame(input_data, ["label", "features"])

columns =  12003


In [7]:
df.show(10)

+-----+--------------------+
|label|            features|
+-----+--------------------+
|    0|[0.0,0.0,0.0,0.0,...|
|    0|[0.0,0.0,0.0,0.0,...|
|    0|[0.0,0.0,2.0,1.0,...|
|    0|[0.0,0.0,1.0,1.0,...|
|    1|[0.0,1.0,1.0,0.0,...|
|    0|[1.0,0.0,1.0,0.0,...|
|    0|[0.0,1.0,1.0,0.0,...|
|    0|[1.0,0.0,1.0,0.0,...|
|    0|[1.0,1.0,1.0,0.0,...|
|    0|[1.0,1.0,1.0,1.0,...|
+-----+--------------------+
only showing top 10 rows



In [ ]:
assempler = VectorAssembler(inputCols=features,outputCol='features')
output = assempler.transform(data)
final_data = output.select('features','label')
#final_data.show(10)

In [ ]:
final_data.show(10)

# CORRELATION

In [ ]:
from pyspark.ml.stat import Correlation
final_data1 = final_data2.select('features')
corr = Correlation.corr(final_data2, "features")
corr = corr.head()[0].toArray()

In [ ]:
snpsRed = []
snpsRed = s.lowCorrelation(corr, threshold=0.7, up=100, down=99)

In [ ]:
snpsRed1 = []
for i in snpsRed:
    snpsRed1.append(features[i])

assempler = VectorAssembler(inputCols=snpsRed1,outputCol='snpsRed')
output = assempler.transform(data)
final_data = output.select('snpsRed','label')
final_data.show(10)

In [11]:
#train_data,test_data = final_data.randomSplit([0.9,0.1],seed=11)

train_data,test_data = df.randomSplit([0.9,0.1],seed=11)

train_data=train_data.withColumnRenamed('TARGET','label')
test_data=test_data.withColumnRenamed('TARGET','label')

#log_reg = LogisticRegression(featuresCol='features',labelCol='label',maxIter=10)
#lr_model = log_reg.fit(train_data)


In [12]:
t1,t2 = train_data.filter('label == 0').randomSplit([0.001,0.1],seed=11)

In [ ]:
t1.count()

In [35]:
#print("train_data = ", train_data.filter('label == 1').count())
print("train_data = ", t1.filter('label == 0').count())

train_data =  19769


In [30]:
#print("final = ",df.count())
#print("test_data = ", test_data.count())
#print("train_data = ", train_data.count())
#print("all = ", test_data.count() + train_data.count())

final =  242176
test_data =  40377
train_data =  201799
all =  242176


In [ ]:
results = lr_model.transform(test_data)
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluate = BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol='label')
AUC = evaluate.evaluate(results)
AUC

In [ ]:
results.show(10)

In [ ]:
'''Param for metric name in evaluation. Supports: - "rmse" (default): root mean squared error - 
"mse": mean squared error - "r2": R^2^ metric - "mae": mean absolute error '''

from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

#lr = LinearRegression(maxIter=maxIteration)
#pipeline = Pipeline(stages=[output, log_reg])
pipeline = Pipeline(stages=[log_reg])
#fit_model = pipeline.fit(train_data)
modelEvaluator=RegressionEvaluator(predictionCol='prediction', labelCol='label')
paramGrid = ParamGridBuilder().build()

crossval = CrossValidator(estimator=log_reg,
                          estimatorParamMaps=paramGrid,
                          evaluator=modelEvaluator,
                          numFolds=10)

cvModel = crossval.fit(train_data)
cross_results = cvModel.transform(test_data)
results = cross_results.select('prediction','label')
#results.show(10)

In [ ]:
cross_results

In [ ]:
from pyspark.mllib.evaluation import MulticlassMetrics


c = cross_results.rdd
predictionAndLabels = c.map(lambda lp: (lp.prediction, float(lp.label)))
metrics = MulticlassMetrics(predictionAndLabels)


In [ ]:
metrics.confusionMatrix()

In [ ]:
print("accuracy = ", metrics.accuracy)
print("recal = ", metrics.recall(1))
print("precision = ", metrics.precision(1))
print("f1 = ", metrics.fMeasure(1.0))

In [ ]:
r = results.rdd
r3 = r.collect()

In [ ]:
same0 = 0
same1 = 0
sum0 = 0
sum1 = 0
sumall = 0
for i in r3:
    
    if i[1] == 0:
        sum0 += 1
        if i[0] == 0:
            same0 += 1
    elif i[1] == 1:
        sum1 += 1
        if i[0] == 1:
            same1 += 1
    sumall += 1

In [ ]:
print('sum0 = ', sum0)
print('sum1 = ', sum1)
print('same0 = ', same0)
print('same1 = ', same1)
print('all = ', sumall)
print('all2 = ', sum0+sum1)